In [3]:
from sources import *
from analysis import *

### Wishlist
* Optimize program to run faster
* Sell items automatically (once for 5x items, once for 1-9 items. Set as short, medium, long)
* More visibility into buy list; should also inform making items
* Update sell policy to not undercut myself

In [4]:
# GET AH SCANDATA
# LOG OUT
generate_auction_scandata(verbose=True) # 17 sec
# LOG IN
# CHECK MAILBOX

Auction scandata loaded and cleaned. 27175 records
Auction scandata full repository. 2611678 records with 93 snapshots. Repository has *not* been updated this run


In [ ]:
generate_auction_activity(verbose=True) # 1.5sec
generate_inventory(verbose=True) #0.7 sec

analyse_item_prices(verbose=True) # 0.7 sec
analyse_sales_performance() # 0.9 sec
analyse_item_min_sell_price() #0.5 sec
analyse_sell_data()
#analyse_character_needs()

apply_buy_policy()

In [ ]:
sell_policy = pd.read_parquet('outputs/sell_policy.parquet')
sell_policy

In [ ]:
#(sell_policy['immediate_inv'].apply(lambda x: max(x-5, 0)) / 5).astype(int)

In [ ]:
buy_policy = pd.read_parquet('outputs/buy_policy.parquet')
buy_policy

In [ ]:
apply_sell_policy(sale_number=2, stack_size=5, duration='long', factor=1.6)

In [ ]:
apply_sell_policy(sale_number=5, stack_size=1, duration='medium')

In [ ]:
apply_sell_policy(sale_number=2, stack_size=5, duration='medium')